In [ ]:
import time
# from tqdm import tqdm
import csv
import pandas as pd 
#show all columns
pd.set_option("display.max_columns", None)
import numpy as np

In [ ]:
positive = pd.read_csv('positive_training_v1_2022_02_04.csv')

In [ ]:
len(positive)

In [ ]:
"""
open chromosomes data
"""
chr_df = pd.read_csv('chrm_gene.csv')

In [ ]:
chr_df.head()

In [ ]:
"""
oprn protein lengths
"""
prot_len_df = pd.read_csv('protein_len_total.csv', skipinitialspace = True)

In [ ]:
"""
drop column seq
"""
prot_len_df = prot_len_df.drop(columns=['seq'])

In [ ]:
"""
drop nan
"""
prot_len_df=prot_len_df.dropna()

In [ ]:
prot_len_df

In [ ]:
"""
open cbiomutations and remove whitespace
"""
filt_cBIO_mutations_df_new = pd.read_csv('cbio_cancer_count_final_v1_2022_01_29.csv', low_memory=False, skipinitialspace = True)



In [ ]:
filt_cBIO_mutations_df_new.columns

In [ ]:
cBIO_mutation_filt = filt_cBIO_mutations_df_new.filter(items=['gene_name','mutationType', 'mutproteinPosStart', 'sampleId'])

In [ ]:
cBIO_mutation_filt['mutationType'].unique()

In [ ]:
"""
save as csv for Website
"""

# merge protein lengths with mutations data

In [ ]:
"""
merge strickly
"""
mutations_pro_len_df_filt = pd.merge(filt_cBIO_mutations_df_new, prot_len_df, left_on = ['gene_name'], right_on = ['gene_name'])


In [ ]:
len(mutations_pro_len_df_filt)

In [ ]:
len(mutations_pro_len_df_filt['gene_name'].unique())

In [ ]:
"""
merge how= 'left' to identify genes without accession
"""
mutations_pro_len_df = pd.merge(filt_cBIO_mutations_df_new, prot_len_df, left_on = ['gene_name'], right_on = ['gene_name'], how = 'left')


In [ ]:
mutations_pro_len_df.head(2)

In [ ]:
len(mutations_pro_len_df['gene_name'].unique())

In [ ]:
"""
visulaize NAN values in protein lengths
sort by specific samples
"""
no_pro_len_df = mutations_pro_len_df[mutations_pro_len_df['pro_len'].isna()]
no_pro_len_df = no_pro_len_df.sort_values(by='Specific_Samples', ascending=False)

In [ ]:
no_pro_len_df.head(2)

In [ ]:
len(no_pro_len_df)

In [ ]:
len(no_pro_len_df['gene_name'].unique())

In [ ]:
"""
open ptm_df_hum.csv
"""
ptm_df = pd.read_csv('ptm_df_hum.csv', low_memory= False)

In [ ]:
"""
count number of muattions per mut_count
"""
count_uniq_ptm_df = ptm_df.groupby(['GENE', 'ACC_ID']).size().reset_index(name='number_of_occurance')


In [ ]:
count_uniq_ptm_df.head(2)

In [ ]:
"""
merge
"""
no_pro_ptm_df = pd.merge(count_uniq_ptm_df, no_pro_len_df, left_on = 'GENE', right_on= 'gene_name' )

In [ ]:
no_pro_ptm_df.head()

In [ ]:
len(no_pro_ptm_df)

In [ ]:
no_pro_ptm_df_count = no_pro_ptm_df.groupby(['GENE', 'ACC_ID']).size().reset_index(name='number_of_occurance')

In [ ]:
no_pro_ptm_df_count = no_pro_ptm_df_count.filter(items=['GENE', 'ACC_ID'])

In [ ]:
no_pro_ptm_df_count

In [ ]:
"""
save as csv
"""
cols = no_pro_ptm_df_count.columns.to_list()
no_pro_ptm_df_count.to_csv('remaining_gene_acc.csv', columns= cols, index= False)

# merg chromosomes with mutations_pro_len_df_filt

In [ ]:
mut_pro_len_chr_df = pd.merge(mutations_pro_len_df_filt, chr_df, left_on = 'gene_name', right_on ='gene' )

In [ ]:
"""
save as csv
"""
cols= mut_pro_len_chr_df.columns.to_list()
mut_pro_len_chr_df.to_csv('mutations_for_prediction_final_v1_2022_02_04.csv', columns=cols, index= False)

In [ ]:
len(mut_pro_len_chr_df)

# Import hotspots dataset

In [ ]:
cancer_hotspot_df = pd.read_csv ('hotspots_v2.csv')

In [ ]:
cancer_hotspot_df.head(3)

In [ ]:
"""
filter 
"""
cancer_hotspot_df2 = cancer_hotspot_df.filter(['Hugo_Symbol', 'Amino_Acid_Position', 'Mutation_Count', 'Total_Mutations_in_Gene',
       ], axis= 1)

In [ ]:
cancer_hotspot_df2.head(2)

In [ ]:
"""
check for duplicates
"""
duplicate_row2 = cancer_hotspot_df2[cancer_hotspot_df2.duplicated()]
print(duplicate_row2)

In [ ]:
"""
remove duplicates in amino acide position and total mutation
"""
hotspots_data_no_dup_df4 = cancer_hotspot_df2.drop_duplicates()

In [ ]:
len(hotspots_data_no_dup_df4)

In [ ]:
hotspots_data_no_dup_df4.head(3)

In [ ]:
"""
duplicates
"""
hotspot_df = hotspots_data_no_dup_df4

In [ ]:
"""
there are strings in the amino acid position columns
i.e. 'X307_splice'
"""
hotspot_df['aa_position'] = hotspot_df['Amino_Acid_Position'].str.extract('(\d+)')



In [ ]:
hotspot_df['aa_position']=pd.to_numeric(hotspot_df['aa_position'])


In [ ]:
"""
rename columns
"""
hotspot_df = hotspot_df.rename(columns={"Hugo_Symbol": "gene_name", "Amino_Acid_Position": "Amino_Acid_Position_db", 
                                       "Mutation_Count": "Mutation_Count_db", "Total_Mutations_in_Gene": "Total_Mutations_in_Gene_db"})


In [ ]:
"""
rename columns
"""
filt_cBIO_mutations_df_new = mut_pro_len_chr_df.rename(columns = {"mutproteinPosStart": "aa_position"})

In [ ]:
hotspot_df.head(2)

In [ ]:
hotspot_df.info()

In [ ]:
cancer_hotspot_df.columns

In [ ]:
filt_cBIO_mutations_df_new.head(2)

In [ ]:
filt_cBIO_mutations_df_new.info()

In [ ]:
"""
merge both interactions_df and df_main on Uniprot ID and site
"""
merge_hotspots_df = pd.merge(filt_cBIO_mutations_df_new,hotspot_df, on=['gene_name','aa_position'])



In [ ]:
len(merge_hotspots_df)

In [ ]:
positive_training_df = merge_hotspots_df.loc[merge_hotspots_df['Specific_Patients']>2]

In [ ]:
"""
add a new column(hotspot) which is the Y value
"""
positive_training_df['hotspot']= 1

In [ ]:
positive_training_df.head(3)

In [ ]:
len(positive_training_df)

In [ ]:
"""
save positive_training_df as csv
"""
cols=positive_training_df.columns.to_list()
positive_training_df.to_csv('positive_training_v1_2022_02_04.csv', columns= cols, index= False )

In [ ]:
"""
select negative training set
includes mutations in the genes (That has positive mutations), but not duplicates
"""
"""
Genaral code

# Find Rows in DF1 Which Are Not Available in DF2
df = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

df

# Find Rows in DF2 Which Are Not Available in DF1
df = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']

df
"""
# df1= all_data_new(contains all the values)
# df2 = positive_training_df_new(contains hotspots)

# Find Rows in DF1 Which Are Not Available in DF2
all_negative_data_df_all = filt_cBIO_mutations_df_new.merge(positive_training_df, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']


In [ ]:
all_negative_data_df_all.head(2)

I will create a duplicate of all_negative_data_df_all, so I can access the whole dataset later
For the negative dataset I will select mut_count<10

In [ ]:
all_negative_data_df = all_negative_data_df_all.loc[(all_negative_data_df_all['Specific_Patients'] < 20)]
# all_negative_data_df = all_negative_data_df_all

In [ ]:
all_negative_data_df.head(2)

In [ ]:
len(all_negative_data_df)

In [ ]:
"""
create a list from positive_training_df_new['Gene']
"""
pos_gene_list = positive_training_df['gene_name'].tolist()

In [ ]:
len(pos_gene_list)

In [ ]:
print(pos_gene_list[:7])

In [ ]:
"""remove duplicates"""
pos_gene_list_2 = list(set(pos_gene_list))

In [ ]:
len(pos_gene_list_2)

In [ ]:
print(pos_gene_list_2)

In [ ]:
"""
select all the rows that are similar in genes of positive data 
"""
"""for i in pos_gene_list_2:
    print(i)
    negative_training_set_1 = all_negative_data_df.loc[(all_negative_data_df['Gene'] == i)]
    break
# negative_training_set_1 = all_negative_data_df.loc[(all_negative_data_df['Gene'] == 'gene')]"""

boolean_series = all_negative_data_df['gene_name'].isin(pos_gene_list_2)
negative_trainig_df_1 = all_negative_data_df[boolean_series]

In [ ]:
negative_trainig_df_1.head(3)

In [ ]:
"""
randomly select 995 rows from higher_muts_df that are not in positive_training_df
"""
negative_trainig_df = negative_trainig_df_1.sample(n = len(pos_gene_list))

In [ ]:
"""
count number of muattions per mut_count
"""
mut_count_gene_mut_df = negative_trainig_df.groupby(['Specific_Patients']).size().reset_index(name='number_of_occurance')

In [ ]:
mut_count_gene_mut_df

In [ ]:
"""
add a new column(hotspot) which is the Y value
"""
negative_trainig_df['hotspot']= 0

In [ ]:
negative_trainig_df.head(2)

In [ ]:
positive_training_df.columns

In [ ]:
negative_trainig_df.columns

In [ ]:
"""
same column names as positive training set
"""
columns = positive_training_df.columns.tolist()

In [ ]:
negative_trainig_df = negative_trainig_df.filter(items = columns)

In [ ]:
negative_trainig_df.head(2)

In [ ]:
positive_training_df.head(2)

In [ ]:
len(negative_trainig_df)

In [ ]:
"""
save negative_trainig_df as csv
"""
cols= negative_trainig_df.columns.to_list()
negative_trainig_df.to_csv('negative_trainig_df_v1_2022_02_04.csv', columns= cols, index= False)

In [ ]:
"""
concat dataframes
"""
frames = [positive_training_df, negative_trainig_df]

training_df = pd.concat(frames)

In [ ]:
training_df.head()

In [ ]:
len(training_df)

In [ ]:
cols= training_df.columns.to_list()
training_df.to_csv('training_df_v1_2022_02_04.csv', columns= cols, index= False )